In [6]:
import numpy as np 

In [7]:
y = np.array([[0.],
       [1.],
       [1.]])


MAX_ITERS = 2
GAMMA = 0.1
lambda_ = 1.0


tx = np.array([[2.3, 3.2],
       [1. , 0.1],
       [1.4, 2.3]])

initial_w = np.array([[0.5],
       [1. ]])

In [20]:
x = np.genfromtxt('data/dataset/x_train.csv', delimiter=',', skip_header=1, max_rows=1000) 

x[np.isnan(x)] = 0
y = np.genfromtxt('data/dataset/y_train.csv', delimiter=',', skip_header=1, max_rows=1000) 

In [11]:
def compute_logistic_loss(y, tx, w): 

    pred = np.dot(tx, w)
    sigmoids = 1.0 / (1 + np.exp(-pred))
    loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))
    return sigmoids, loss


def logistic_regression(y, tx, initial_w, max_iters, gamma):
    """
    Perform logistic regression using gradient descent.
    
    Parameters:
    y: np.array
        The target values
    tx: np.array
        The data matrix (each row is a data point)
    initial_w: np.array
        Initial weights
    max_iters: int
        Maximum number of iterations for gradient descent
    gamma: float
        Learning rate

    Returns:
    w: np.array
        Optimized weights after training
    """
    
    w = initial_w

    sigmoids, loss = compute_logistic_loss(y, tx, w)
    
    for iter in range(max_iters):
        # compute the gradient

        print(w[0], loss)
        grad =  tx.T.dot(sigmoids - y)/len(tx)

        # update w through the negative gradient direction
        w = w - gamma * grad

        sigmoids, loss = compute_logistic_loss(y, tx, w)
        
    return w, loss

In [31]:
def compute_logistic_loss(y, tx, w): 

    pred = np.dot(tx, w)
    print(pred)
    sigmoids = 1.0 / (1 + np.exp(-pred))

    loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))
    return sigmoids, loss

N, D = x.shape
compute_logistic_loss(y, x, np.random.rand(D) * 0.001)

[2716878.66256325 2717165.9771591  2716439.81422924 2714982.82935463
 2716753.93231744 2714312.68426805 2717154.67590987 2714292.34211474
 2715393.75885071 2714909.23792666 2717143.02974004 2713534.58788835
 2715342.19125035 2716441.2299616  2713515.99804771 2714594.08570406
 2715475.77013581 2715690.83311691 2715683.11639031 2716775.97826806
 2717207.13363592 2715579.86300597 2715282.04778189 2716823.29836274
 2714988.52793951 2713219.06750504 2717222.58071048 2715403.83196101
 2715696.41499938 2716043.63536271 2714286.67321047 2713881.1594835
 2715708.78500708 2715212.41551501 2714576.75431277 2713476.7271976
 2717226.04421431 2713488.59986211 2714950.88076736 2715652.51308143
 2713593.29783936 2715993.29778921 2713577.18688556 2713650.88579147
 2716022.3655408  2716099.75995822 2714581.57130953 2715250.79367197
 2713136.92142605 2713899.44960621 2715777.37601548 2714586.17901068
 2716764.8844781  2715404.98387339 2715587.99105388 2715760.2354908
 2714261.57197939 2713200.52458996 27

/tmp/ipykernel_8600/3118247014.py:7: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))
/tmp/ipykernel_8600/3118247014.py:7: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))


(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 

In [27]:


N, D = x.shape
initial_w = np.random.rand(D) * 0.0000001

w, loss = logistic_regression(y, x, initial_w, 100, 0.4)
print(w, loss)

/tmp/ipykernel_8600/3216979147.py:6: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))
/tmp/ipykernel_8600/3216979147.py:6: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))
/tmp/ipykernel_8600/3216979147.py:5: RuntimeWarning: overflow encountered in exp
  sigmoids = 1.0 / (1 + np.exp(-pred))


8.086226799182496e-08 nan
-11.166799919137734 nan
-10.052799919137733 nan
-8.938799919137733 nan
-7.824799919137733 nan
-6.710799919137733 nan
-5.596799919137733 nan
-4.482799919137733 nan
-3.368799919137733 nan
-2.2547999191377333 nan
-1.1407999191377332 nan
-0.026799919137733053 nan
-11.193599919137736 nan
-10.079599919137735 nan
-8.965599919137734 nan
-7.851599919137734 nan
-6.737599919137734 nan
-5.623599919137734 nan
-4.5095999191377345 nan
-3.3955999191377346 nan
-2.2815999191377347 nan
-1.1675999191377346 nan
-0.05359991913773454 nan
-11.220399919137737 nan
-10.106399919137736 nan
-8.992399919137736 nan
-7.878399919137736 nan
-6.764399919137736 nan
-5.650399919137736 nan
-4.536399919137736 nan
-3.422399919137736 nan
-2.3083999191377362 nan
-1.1943999191377361 nan
-0.08039991913773603 nan
-11.247199919137739 nan
-10.133199919137738 nan
-9.019199919137737 nan
-7.905199919137737 nan
-6.791199919137737 nan
-5.677199919137737 nan
-4.5631999191377375 nan
-3.4491999191377376 nan
-2.335

-11.27399991913774 nan
-10.15999991913774 nan
-9.045999919137738 nan
-7.931999919137739 nan
-6.817999919137739 nan
-5.703999919137739 nan
-4.589999919137739 nan
-3.475999919137739 nan
-2.361999919137739 nan
-1.247999919137739 nan
-0.133999919137739 nan
-11.300799919137742 nan
-10.18679991913774 nan
-9.07279991913774 nan
-7.95879991913774 nan
-6.84479991913774 nan
-5.73079991913774 nan
-4.6167999191377405 nan
-3.5027999191377406 nan
-2.3887999191377407 nan
-1.2747999191377406 nan
-0.1607999191377405 nan
-11.327599919137743 nan
-10.213599919137742 nan
-9.099599919137741 nan
-7.985599919137742 nan
-6.871599919137742 nan
-5.757599919137742 nan
-4.643599919137742 nan
-3.529599919137742 nan
-2.415599919137742 nan
-1.301599919137742 nan
-0.187599919137742 nan
-11.354399919137744 nan
-10.240399919137744 nan
-9.126399919137743 nan
-8.012399919137742 nan
-6.898399919137742 nan
-5.784399919137742 nan
-4.6703999191377426 nan
-3.5563999191377427 nan
-2.442399919137743 nan
-1.3283999191377427 nan
-1